In [70]:
import configparser
from operator import le
import snowflake.connector
import pandas as pd

def getProp():
	global config
	config = configparser.RawConfigParser()
	config.read('../configuration.properties')
	return True

# Function to fetch data from Snowflake
def fetch_data_from_snowflake():
    try:
        getProp()
        conn = snowflake.connector.connect(
            user=config['SNOWFLAKE']['Username'],
            password=config['SNOWFLAKE']['Password'],
            account=config['SNOWFLAKE']['Account'],
            warehouse=config['SNOWFLAKE']['Warehouse'],
            database=config['SNOWFLAKE']['Database'],
            schema=config['SNOWFLAKE']['Schema']
        )

        cur = conn.cursor()

        sql = "SELECT * FROM CFA_WEBSITE_DATA"
        cur.execute(sql)
        df = cur.fetchall()

    except Exception as e:
        print(e)

    return df

# Function to fetch data from Snowflake
df = fetch_data_from_snowflake()
print(df)

df = pd.read_csv('../data/input/scraped_data.csv', sep = '\t')
df.head()

[]


,topic_name,year,level,introduction,learning_outcome,summary,summary_page_link,pdf_file_link
0,Time-Series Analysis,2024.0,II,"As financial analysts, we often use time-seri...",The member should be able to: calculate and...,The predicted trend value of a time series in...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
1,Credit Analysis Models,2024.0,II,Credit analysis plays an important role in th...,The member should be able to: explain expec...,This reading has covered several important top...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
2,Introduction to Alternative Investments,2023.0,I,"In this section, we explain what alternative ...",The member should be able to: describe type...,This reading provides a comprehensive introduc...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
3,Credit Default Swaps,2024.0,II,Derivative instruments in which the underlyin...,The member should be able to: describe cred...,A credit default swap (CDS) is a contract betw...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
4,Valuation of Contingent Claims,2024.0,II,A contingent claim is a derivative instrument...,The member should be able to: describe and ...,This reading on the valuation of contingent cl...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...


In [72]:
from venv import create
from tomlkit import table


def filter_dataframe(df, topic_names: list):
    '''
    # Filter the dataframe for the specified topic names
    topic_names = [
    "Overview of Equity Securities",
    "Market Organization and Structure",
    "Security Market Indexes"
    ]
    '''

    filtered_df = df[df['topic_name'].isin(topic_names)]

    # Remove the specified line from learning outcomes and expand into separate rows
    expanded_list_cleaned = []
    for index, row in filtered_df.iterrows():
        learning_outcomes = row['learning_outcome'].replace(
            "The member should be able to:", "").split(";")
        for outcome in learning_outcomes:
            cleaned_outcome = outcome.strip()
            if cleaned_outcome:  # Ensure the outcome is not empty
                expanded_list_cleaned.append({
                    "topic_name": row['topic_name'],
                    "learning_outcome": cleaned_outcome,
                    "summary": row['summary']
                })

    # Recreate the expanded dataframe without the unnecessary line
    new_df = pd.DataFrame(expanded_list_cleaned)

    return new_df


df = filter_dataframe(df, topic_names=["Overview of Equity Securities",
                                       "Market Organization and Structure",
                                       "Security Market Indexes"])
print(df)

                           topic_name  \
0   Market Organization and Structure   
1   Market Organization and Structure   
2   Market Organization and Structure   
3   Market Organization and Structure   
4   Market Organization and Structure   
5   Market Organization and Structure   
6   Market Organization and Structure   
7   Market Organization and Structure   
8   Market Organization and Structure   
9   Market Organization and Structure   
10  Market Organization and Structure   
11  Market Organization and Structure   
12            Security Market Indexes   
13            Security Market Indexes   
14            Security Market Indexes   
15            Security Market Indexes   
16            Security Market Indexes   
17            Security Market Indexes   
18            Security Market Indexes   
19            Security Market Indexes   
20            Security Market Indexes   
21            Security Market Indexes   
22            Security Market Indexes   
23      Overview

True

In [74]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
import os

def query_openai_for_technical_document(learning_outcome, summary, api_key):

    # completion llm
    llm = ChatOpenAI(
        openai_api_key=api_key,
        model_name='gpt-3.5-turbo',
        temperature=0.0,
        max_tokens=2048,
        frequency_penalty=0.1,
        presence_penalty=0.1,
    )

    messages = [
        SystemMessage(content="You are provided a SUMMARY below. Your goal is to provide a technical note that relates to the LEARNING OUTCOME. Include tables, figures and equations, ONLY if present in the CONTEXT. Ensure that you stickly stay within the CONTEXT and use no outside information. Answer in less than 300 words. No heading is needed. \LEARNING OUTCOMES:\n" + learning_outcome + "\n\nSUMMARY:\n" + summary + "\n\n"),
    ]

    response = llm.invoke(messages)
    response = response.content

    return response



In [79]:
# Function to query OpenAI and update the dataframe
def generate_technical_documents(df):

    getProp()
    api_key = config['OPENAI']['OPENAI_API_KEY']
    # Append a blank "Technical Document" column
    df["technical_document"] = ""

    for index, row in df.iterrows():

        # Query OpenAI for each row and update the 'Technical Document' column
        df.at[index, 'technical_document'] = query_openai_for_technical_document(
            learning_outcome=row['learning_outcome'],
            summary=row['summary'],
            api_key=api_key
        )

    try:
        getProp()
        conn = snowflake.connector.connect(
            user=config['SNOWFLAKE']['Username'],
            password=config['SNOWFLAKE']['Password'],
            account=config['SNOWFLAKE']['Account'],
            warehouse=config['SNOWFLAKE']['Warehouse'],
            database=config['SNOWFLAKE']['Database'],
            schema=config['SNOWFLAKE']['Schema']
        )
        filtered_table_name = config['SNOWFLAKE']['Filtered_Table_Name']

        cur = conn.cursor()

        # Create the table if it does not exist
        create_table_sql = f"CREATE OR REPLACE TABLE {filtered_table_name} (topic_name VARCHAR, learning_outcome VARCHAR, summary VARCHAR, technical_document VARCHAR)"
        cur.execute(create_table_sql)

        insert_data_sql = f"INSERT INTO {filtered_table_name} (topic_name, learning_outcome, summary, technical_document) VALUES (%s, %s, %s, %s)"

        # Iterate through the DataFrame and insert each row
        for index, row in df.iterrows():
            # Parameters to insert for each row
            data_tuple = (row['topic_name'],
                          row['learning_outcome'], row['summary'], row['technical_document'])
            # Execute the insert command with the parameters
            cur.execute(insert_data_sql, data_tuple)

    except Exception as e:
        print(e)
    
    df = df.drop(columns=['summary'])
    df.to_csv('../data/output/technical_documents.csv', index=False)

    return df

final_df = generate_technical_documents(df)
final_df

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what 

,topic_name,learning_outcome,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,The main functions of the financial system inc...
1,Market Organization and Structure,describe classifications of assets and markets,The classification of assets and markets is cr...
2,Market Organization and Structure,"describe the major types of securities, curren...","The major types of securities, currencies, con..."
3,Market Organization and Structure,describe types of financial intermediaries and...,Financial intermediaries play a crucial role i...
4,Market Organization and Structure,compare positions an investor can take in an a...,The financial system allows investors to take ...
5,Market Organization and Structure,"calculate and interpret the leverage ratio, th...",The leverage ratio is a key concept in margin ...
6,Market Organization and Structure,"compare execution, validity, and clearing inst...",The reading discusses the importance of unders...
7,Market Organization and Structure,compare market orders with limit orders,Market orders and limit orders are two common ...
8,Market Organization and Structure,define primary and secondary markets and expla...,The primary market is where securities are fir...
9,Market Organization and Structure,"describe how securities, contracts, and curren...","Securities, contracts, and currencies are trad..."


In [30]:
final_df.head()

,topic_name,learning_outcome,summary,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,This reading introduces how the financial syst...,The main functions of the financial system inc...
1,Market Organization and Structure,describe classifications of assets and markets,This reading introduces how the financial syst...,The classification of assets and markets is cr...
2,Market Organization and Structure,"describe the major types of securities, curren...",This reading introduces how the financial syst...,"The major types of securities, currencies, con..."
3,Market Organization and Structure,describe types of financial intermediaries and...,This reading introduces how the financial syst...,Financial intermediaries play a crucial role i...
4,Market Organization and Structure,compare positions an investor can take in an a...,This reading introduces how the financial syst...,The financial system allows investors to take ...


In [53]:
final_df = final_df.drop(columns=['summary'])
final_df.head()

,topic_name,learning_outcome,technical_document
0,Market Organization and Structure,explain the main functions of the financial sy...,The main functions of the financial system inc...
1,Market Organization and Structure,describe classifications of assets and markets,The classification of assets and markets is cr...
2,Market Organization and Structure,"describe the major types of securities, curren...","The major types of securities, currencies, con..."
3,Market Organization and Structure,describe types of financial intermediaries and...,Financial intermediaries play a crucial role i...
4,Market Organization and Structure,compare positions an investor can take in an a...,The financial system allows investors to take ...


In [54]:
final_df.to_csv('../data/output/technical_documents.csv', index=False)

In [51]:
def generate_markdown_file(final_df):
# Grouping the data by topic_name and creating the markdown content
    markdown_content = ""
    for topic_name, group in final_df.groupby("topic_name"):
        markdown_content += f"## {topic_name}\n\n"
        for _, row in group.iterrows():
            markdown_content += f"# Learning Outcome:\n {row['learning_outcome']}\n"
            markdown_content += f"# Technical Document:\n {row['technical_document']}\n\n"

    # Saving the markdown content to a .txt file
    markdown_file_path = '../data/output/technical_documents_markdown.md'
    with open(markdown_file_path, 'w') as file:
        file.write(markdown_content)

    return markdown_file_path

'../data/output/technical_documents_markdown.md'